>[MLCC Week 1 Task](#scrollTo=toNrmbxxHLOL)

>[Step 1: Getting the data](#scrollTo=PikbkmYRH_Ls)

>[Step 2: Cleaning the data](#scrollTo=Yq6B0BseIhHj)

>[Step 3: Configuring the LinearRegressor](#scrollTo=-deOxlscJ_ud)

>[Task 1: Get the loss less than 15000](#scrollTo=1ce2H07bKKxm)

>[Task 2: Take another feature](#scrollTo=inRwqESaLqiE)



# MLCC Week 1 Task

Online news websites play an important role in our day to day life. Not only we read news online, we also share it as per our liking.

Just like the cover of a book makes the first impression, the title of the news can have a lasting effect on the popularity of the news

In this task, we will be using a database of online news popularity provided by UCL. Your task is to predict how many **shares** an online news will get depending on the **number of tokens** (roughly speaking, tokens are words that are important) in its title.

# Step 1: Getting the data

The file provided by UCL is a zip file. So we will use Python magic to unzip and take the csv

We use the [wget module](https://pypi.org/project/wget/) to download the file and [zipfile module](https://docs.python.org/3/library/zipfile.html) to unzip

In [0]:
! pip install wget # The ! before suggests it's a shell command. We install wget module to ease the download process
import wget
filename = wget.download('https://archive.ics.uci.edu/ml/machine-learning-databases/00332/OnlineNewsPopularity.zip') # filename stores the name of the downloaded file

In [0]:
print(filename)

Now we use the zipfile module provided by Python to unzip the file in current directory.

In [0]:
import zipfile
zip_ref = zipfile.ZipFile(filename, 'r')
zip_ref.extractall('.')

Let's list all the files in the extracted directory. Make sure there is one csv and one names file

In [0]:
! ls OnlineNewsPopularity/

# Step 2: Cleaning the data

Now we move on to our usual routine. First, all the imports

In [0]:
import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

Now we read the file in the dataframe.

In [0]:
popularity_dataframe = pd.read_csv('OnlineNewsPopularity/OnlineNewsPopularity.csv')
popularity_dataframe.describe()

Don't foreget to randomize the data. 

For simplicity and faster result, we drop any rows which have more than 500 shares

We also scale the "shares" column by 100 so that's they're spaced (somewhat) apart and are easy to visualized. This makes our data in the range of 50000. So we should expect our RMSE to be measured in the same scale.

**N.B.** The column names have an empty space before them. Tensorflow doesn't like having a space in its scope name. So we change the name of the column

In [0]:
popularity_dataframe.reindex(np.random.permutation(popularity_dataframe.index))
_ = popularity_dataframe[' shares'] # Look closely! There's a space in the beginning
popularity_dataframe = popularity_dataframe.drop(popularity_dataframe[popularity_dataframe[' shares'] > 500].index) # Drop the rows with more than 500 shares
popularity_dataframe['normalized_shares'] = popularity_dataframe[' shares'] * 100 # Scale up by 100
popularity_dataframe.rename(columns={' n_tokens_title': 'n_tokens_title'}, inplace=True) # Rename the column and remove the space
popularity_dataframe.describe()

Our feature column would be the **n_tokens_title** column and target would be the **normalized_shares column**. Let's plot these two.

In [0]:
X = popularity_dataframe[['n_tokens_title']]
Y = popularity_dataframe['normalized_shares']
plt.scatter(X, Y)
plt.show()

# Step 3: Configuring the LinearRegressor

Now as usual, we create the feature columns and create the LinearRegressor.

We start by defining the input function which we have used before

In [0]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a linear regression model of one feature.
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
    
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                           

    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)

    # Shuffle the data, if specified.
    if shuffle:
      ds = ds.shuffle(buffer_size=10000)
      
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()

    return features, labels

The rest of the routine jobs of configuring the regressor, training and predicting have been wrapped in this convenient function which can be called with the parameters learning rate, steps, batch size and feature column

In [0]:
def train_model(learning_rate, steps, batch_size, input_feature="n_tokens_title"):
  """Trains a linear regression model of one feature.
  
  Args:
    learning_rate: A `float`, the learning rate.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    input_feature: A `string` specifying a column from `popularit_dataframe`
      to use as input feature.
  """
  
  periods = 10
  steps_per_period = steps / periods

  my_feature = input_feature
  my_feature_data = popularity_dataframe[[my_feature]]
  my_label = "normalized_shares"
  targets = popularity_dataframe[my_label]

  # Create feature columns.
  feature_columns = [tf.feature_column.numeric_column(my_feature)]
  
  # Create input functions.
  training_input_fn = lambda:my_input_fn(my_feature_data, targets, batch_size=batch_size)
  prediction_input_fn = lambda: my_input_fn(my_feature_data, targets, num_epochs=1, shuffle=False)
  
  # Create a linear regressor object.
  my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  linear_regressor = tf.estimator.LinearRegressor(
      feature_columns=feature_columns,
      optimizer=my_optimizer
  )

  # Set up to plot the state of our model's line each period.
  plt.figure(figsize=(15, 6))
  plt.subplot(1, 2, 1)
  plt.title("Learned Line by Period")
  plt.ylabel(my_label)
  plt.xlabel(my_feature)
  sample = popularity_dataframe.sample(n=300)
  plt.scatter(sample[my_feature], sample[my_label])
  colors = [cm.coolwarm(x) for x in np.linspace(-1, 1, periods)]

  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print("Training model...")
  print("RMSE (on training data):")
  root_mean_squared_errors = []
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    linear_regressor.train(
        input_fn=training_input_fn,
        steps=steps_per_period
    )
    # Take a break and compute predictions.
    predictions = linear_regressor.predict(input_fn=prediction_input_fn)
    predictions = np.array([item['predictions'][0] for item in predictions])
    
    # Compute loss.
    root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(predictions, targets))
    # Occasionally print the current loss.
    print("  period %02d : %0.2f" % (period, root_mean_squared_error))
    # Add the loss metrics from this period to our list.
    root_mean_squared_errors.append(root_mean_squared_error)
    # Finally, track the weights and biases over time.
    # Apply some math to ensure that the data and line are plotted neatly.
    y_extents = np.array([0, sample[my_label].max()])
    
    weight = linear_regressor.get_variable_value('linear/linear_model/%s/weights' % input_feature)[0]
    bias = linear_regressor.get_variable_value('linear/linear_model/bias_weights')

    x_extents = (y_extents - bias) / weight
    x_extents = np.maximum(np.minimum(x_extents,
                                      sample[my_feature].max()),
                           sample[my_feature].min())
    y_extents = weight * x_extents + bias
    plt.plot(x_extents, y_extents, color=colors[period]) 
  print("Model training finished.")

  # Output a graph of loss metrics over periods.
  plt.subplot(1, 2, 2)
  plt.ylabel('RMSE')
  plt.xlabel('Periods')
  plt.title("Root Mean Squared Error vs. Periods")
  plt.tight_layout()
  plt.plot(root_mean_squared_errors)

  # Output a table with calibration data.
  calibration_data = pd.DataFrame()
  calibration_data["predictions"] = pd.Series(predictions)
  calibration_data["targets"] = pd.Series(targets)
  display.display(calibration_data.describe())

  print("Final RMSE (on training data): %0.2f" % root_mean_squared_error)

Now we can call the function

In [0]:
train_model(
    learning_rate=0.1,
    steps=100,
    batch_size=1
)

# Task 1: Get the loss less than 15000

Tweak the learning rate until you have a loss less than 15000
**Hint**:


1.   Check the initial few lines. If they are too close, increase the learning rate. If they are too sparse, decrease it.
2.   The loss function should decrease steeply at first, and then decrease slowly.
3. If the training has not converged, try running it for longer. Change the **steps** parameter for that
4. If the loss function first decreases and then increases, it means you have overshot the minima. Try decreasing the learning rate.
5. Very small batch size can cause problem, and so can very large batch size.



# Task 2: Take another feature

Take any other feature and see if you can do better or not.

Remember, there might be spaces in front of the column name. Make sure to rename the columns, if needed

In [0]:
# Rename the columns if necessary
# popularity_dataframe.rename(...) 
train_model(
    learning_rate=0.1,
    steps=100,
    batch_size=1,
    input_feature = # YOUR CODE HERE
)

Created by Aniket Bhattacharyea (aniketmail669@gmail.com)